In [31]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datetime import datetime as dt
from prophet import Prophet

try:
    import neuralprophet
except:
    !pip install neuralprophet
    import neuralprophet
    
from neuralprophet import NeuralProphet

In [32]:
url = 'https://cexa.oceanprotocol.io/ohlc?exchange=binance&pair=ETH/USDT&period=1h'
df = pd.read_json(url)
df.columns = ['dt1', 'open', 'high', 'low', 'close', 'volume']

df['dt1'] = df.dt1.apply(lambda x: dt.utcfromtimestamp(x/1000))
df.set_index('dt1', inplace = True)

In [33]:
x

,ds,y
0,2023-02-23 04:00:00,1667.44
1,2023-02-23 05:00:00,1671.49
2,2023-02-23 06:00:00,1664.83
3,2023-02-23 07:00:00,1665.51
4,2023-02-23 08:00:00,1666.70
...,...,...
994,2023-04-05 15:00:00,1897.46
995,2023-04-05 16:00:00,1897.21
996,2023-04-05 17:00:00,1893.28
997,2023-04-05 18:00:00,1898.14


In [34]:
# 1) prepare dataframe
x = df.open
x = x.reset_index()
x.columns = ['ds','y']

# 2) build model
m = Prophet(weekly_seasonality=True, daily_seasonality=True, )
m.fit(x)

# # 3) predict
t1 = 15
df1 = m.make_future_dataframe(periods=t1, freq='h')
df_out = m.predict(df1)
df_out

19:29:55 - cmdstanpy - INFO - Chain [1] start processing
19:29:56 - cmdstanpy - INFO - Chain [1] done processing


,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,daily,daily_lower,daily_upper,weekly,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2023-02-23 04:00:00,1640.799978,1628.007970,1686.537037,1640.799978,1640.799978,17.192004,17.192004,17.192004,-0.382524,-0.382524,-0.382524,17.574528,17.574528,17.574528,0.0,0.0,0.0,1657.991982
1,2023-02-23 05:00:00,1640.488248,1625.464466,1687.582127,1640.488248,1640.488248,15.932097,15.932097,15.932097,-0.645830,-0.645830,-0.645830,16.577927,16.577927,16.577927,0.0,0.0,0.0,1656.420345
2,2023-02-23 06:00:00,1640.176518,1628.562133,1686.716129,1640.176518,1640.176518,15.490028,15.490028,15.490028,-0.099583,-0.099583,-0.099583,15.589611,15.589611,15.589611,0.0,0.0,0.0,1655.666546
3,2023-02-23 07:00:00,1639.864788,1624.028568,1686.253147,1639.864788,1639.864788,15.069480,15.069480,15.069480,0.452490,0.452490,0.452490,14.616990,14.616990,14.616990,0.0,0.0,0.0,1654.934268
4,2023-02-23 08:00:00,1639.553058,1624.661307,1685.701307,1639.553058,1639.553058,13.915765,13.915765,13.915765,0.248792,0.248792,0.248792,13.666974,13.666974,13.666974,0.0,0.0,0.0,1653.468824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1009,2023-04-06 06:00:00,1874.140047,1860.017701,1921.527801,1873.516046,1874.737823,15.490028,15.490028,15.490028,-0.099583,-0.099583,-0.099583,15.589611,15.589611,15.589611,0.0,0.0,0.0,1889.630075
1010,2023-04-06 07:00:00,1874.694565,1858.804378,1917.306352,1873.760133,1875.691539,15.069480,15.069480,15.069480,0.452490,0.452490,0.452490,14.616990,14.616990,14.616990,0.0,0.0,0.0,1889.764045
1011,2023-04-06 08:00:00,1875.249083,1857.601194,1920.034194,1873.843061,1876.616345,13.915765,13.915765,13.915765,0.248792,0.248792,0.248792,13.666974,13.666974,13.666974,0.0,0.0,0.0,1889.164849
1012,2023-04-06 09:00:00,1875.803601,1857.705990,1918.748551,1874.182821,1877.724225,12.194685,12.194685,12.194685,-0.551225,-0.551225,-0.551225,12.745910,12.745910,12.745910,0.0,0.0,0.0,1887.998287


In [35]:
# 2) build model
m = NeuralProphet()
m.fit(x)

WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.8% of the data.
INFO - (NP.df_utils._infer_frequency) - Dataframe freq automatically defined as H
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.
INFO - (NP.utils.set_auto_seasonalities) - Disabling yearly seasonality. Run NeuralProphet with yearly_seasonality=True to override this.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 16
INFO - (NP.config.set_auto_batch_epoch) - Auto-set epochs to 194
WARNING - (NP.config.set_lr_finder_args) - Learning rate finder: The number of batches (63) is too small than the required number for the learning rate finder (226). The results might not be optimal.


Finding best initial lr:   0%|          | 0/226 [00:00<?, ?it/s]

Training: 0it [00:00, ?it/s]

,MAE,RMSE,Loss,RegLoss,epoch
0,495.468842,612.555847,0.501212,0.0,0
1,472.617218,585.356079,0.467154,0.0,1
2,450.789062,562.250244,0.432749,0.0,2
3,420.704956,524.111877,0.388247,0.0,3
4,386.615540,481.725708,0.339080,0.0,4
...,...,...,...,...,...
189,22.784264,28.969151,0.001644,0.0,189
190,22.607315,28.856359,0.001622,0.0,190
191,22.663910,29.063679,0.001628,0.0,191
192,22.673429,28.872816,0.001630,0.0,192


In [43]:
future = m.make_future_dataframe(x, periods=15, n_historic_predictions=len(x)-50)
forecast = m.predict(future)

INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.8% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.793% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.793% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H


Predicting: 63it [00:00, ?it/s]

INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column


In [54]:
forecast.tail(25)

,ds,y,yhat1,trend,season_weekly,season_daily
939,2023-04-05 10:00:00,1909.29,1881.049561,1851.381226,30.472904,-0.804605
940,2023-04-05 11:00:00,1913.87,1882.710571,1851.775513,30.483778,0.451268
941,2023-04-05 12:00:00,1914.56,1884.750244,1852.169800,30.365213,2.215217
942,2023-04-05 13:00:00,1921.73,1886.362061,1852.564209,30.122736,3.675260
943,2023-04-05 14:00:00,1912.91,1887.274292,1852.958496,29.762896,4.553024
944,2023-04-05 15:00:00,1897.46,1886.663574,1853.352661,29.293201,4.017626
945,2023-04-05 16:00:00,1897.21,1884.296143,1853.746948,28.721991,1.827216
946,2023-04-05 17:00:00,1893.28,1882.171265,1854.141235,28.058388,-0.028314
947,2023-04-05 18:00:00,1898.14,1882.092163,1854.535522,27.312153,0.244453
948,2023-04-05 19:00:00,1900.13,1882.314209,1854.929810,26.493580,0.890873


In [69]:
forecast[-13:-1].yhat1

951    1874.168335
952    1875.689819
953    1877.628296
954    1878.021851
955    1877.741699
956    1877.443848
957    1876.338623
958    1874.659058
959    1874.005005
960    1874.471191
961    1874.349854
962    1873.250366
Name: yhat1, dtype: float64

In [70]:
x_out = forecast["yhat1"][-13:-1]

In [71]:
x_out

951    1874.168335
952    1875.689819
953    1877.628296
954    1878.021851
955    1877.741699
956    1877.443848
957    1876.338623
958    1874.659058
959    1874.005005
960    1874.471191
961    1874.349854
962    1873.250366
Name: yhat1, dtype: float64

In [72]:
x_out = str(list(x_out.map(lambda x: round(x,3)))).replace(' ','')
x_out

'[1874.168,1875.69,1877.628,1878.022,1877.742,1877.444,1876.339,1874.659,1874.005,1874.471,1874.35,1873.25]'